## 💬 Chat Templates: Use Cases Overview

In this Colab, we’ll implement four distinct use cases to explore different capabilities of language models:

| Use Case | Task                          | Description                                                                 |
|----------|-------------------------------|-----------------------------------------------------------------------------|
| 1        | Classification                | Label the intent of a sentence (e.g., positive/negative, task/info/complaint) |
| 2        | Conversational Chat           | Generate simple customer support responses in a chat format                |
| 3        | Extend Context Size           | Simulate longer context handling by padding a prompt with additional text  |
| 4        | Multi-Dataset Single Finetuning | Finetune the model using a combination of two small datasets               |


In [1]:
# Install dependencies
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

# 🧪 Multi-Task Finetuning with Chat Templates (Classification, Chat, Long Context)

In [2]:
import torch
from datasets import Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling


# 🧠 Load the Base Model

In [3]:
# Load model
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# 🏷️ Use Case 1: Intent Classification Dataset

In [4]:
# ------------------------------
# 1️⃣ Dataset 1 — Classification-style (intent labeling)
# ------------------------------
classification_texts = [
    "User: I can't access my account.\nAssistant:",
    "User: What are your operating hours?\nAssistant:",
    "User: Your service is terrible!\nAssistant:",
    "User: Thanks for the help!\nAssistant:",
]
classification_labels = [
    " complaint", " information", " complaint", " praise"
]

dataset1 = Dataset.from_dict({
    "text": [q + a for q, a in zip(classification_texts, classification_labels)]
})

# 💬 Use Case 2: Conversational Chat Dataset

In [5]:
# ------------------------------
# 2️⃣ Dataset 2 — Conversational chat-style
# ------------------------------
chat_texts = [
    "User: Hello, can I book a table for 2 tomorrow night?\nAssistant: Sure, I can help you with that.",
    "User: Do you have vegetarian options?\nAssistant: Yes, we offer a variety of vegetarian dishes.",
    "User: How do I cancel my booking?\nAssistant: You can cancel it from the booking confirmation email.",
]

dataset2 = Dataset.from_dict({"text": chat_texts})

# 📄 Use Case 3: Extended Context Simulation Dataset

In [6]:
# ------------------------------
# 3️⃣ Extended context simulation
# ------------------------------
long_prompt = "User: " + "Tell me a story. " * 20 + "\nAssistant:"
long_answer = " Once upon a time, in a land far away, there lived a talking parrot..."
dataset3 = Dataset.from_dict({"text": [long_prompt + long_answer]})

# 🔄 Use Case 4: Combine All Datasets into a Unified Training Set

In [7]:
# ------------------------------
# 4️⃣ Combine all datasets for multi-source training
# ------------------------------
all_datasets = concatenate_datasets([dataset1, dataset2, dataset3])

# Tokenize dataset
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = all_datasets.map(tokenize_fn, batched=False)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

# ⚙️ Setup Training Arguments

In [8]:
# ------------------------------
# Training setup (CPU friendly)
# ------------------------------
training_args = TrainingArguments(
    output_dir="./chat-template-finetune",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=1,
    save_steps=10,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# 🚀 Train Model on Multiple Chat Template Tasks

In [9]:
# Train on all chat + classification formats
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,3.640300
2,4.627700
3,4.176500
4,1.036500
5,4.840100
6,4.430900
7,3.406800
8,3.253000


TrainOutput(global_step=8, training_loss=3.676463335752487, metrics={'train_runtime': 32.3966, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.247, 'total_flos': 261296750592.0, 'train_loss': 3.676463335752487, 'epoch': 1.0})